In [79]:
from engine import PatentRetrievalService

In [80]:
service = PatentRetrievalService('data.txt')


Batches: 100%|██████████| 149/149 [01:59<00:00,  1.25it/s]


In [81]:
results = service.retrieve_patents(
    keywords=["enzymes"],
    precision_recall_balance=0.99999
)

In [82]:
results['retrieved_patents']

[{'abstract': 'Ein Blockchain-System (100) für transparentes Lieferkettenmanagement, wobei das System (100) Folgendes umfasst:  eine Datenerfassungseinheit (102) zum Erfassen einer Kombination aus mehreren Eingabedaten, die auf einem Produkt von einem Lieferanten basieren und in einer Blockchain (102a) gespeichert sind;  ein Internet der Dinge (IoT)-Sensor (104), der in einem Produktionsgerät und einem Lagerbereich eingebettet ist, um Temperatur und Luftfeuchtigkeit während des gesamten Herstellungsprozesses kontinuierlich zu überwachen und in der Blockchain (102a) gespeichert ist;  eine Verarbeitungseinheit (106) zum Verarbeiten der erfassten Daten, um die Datenelemente zu identifizieren, die mittels eines Smart Contracts in einem Blockchain-Netzwerk erfasst werden sollen, wobei der Smart Contract einen Vergleich und eine Überprüfung der Eingabedaten auf der Grundlage vorgespeicherter Daten durchführt, um die Authentizität der Eingabedaten zu bewerten und die Gesamtleistung des Liefer

In [51]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
model_name = 'paraphrase-multilingual-MiniLM-L12-v2'

dataset_path = 'data.txt'
with open(dataset_path, 'r', encoding='utf-8') as f:
        abstracts = [line.strip() for line in f if line.strip()]

embedding_model = SentenceTransformer(model_name)

In [ ]:
embeddings = embedding_model.encode(sentences=abstracts, show_progress_bar=True)

Batches: 100%|██████████| 149/149 [02:00<00:00,  1.24it/s]


In [23]:
faiss.normalize_L2(embeddings)

In [34]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product index for cosine similarity
index.add(embeddings)

In [55]:
keyword = ["enzymes", "medical"]
keyword_embedding = embedding_model.encode(keyword)

In [56]:
faiss.normalize_L2(keyword_embedding)

In [57]:
distances, indices = index.search(keyword_embedding, k=len(abstracts))

In [60]:
avg_similarities = np.mean(distances, axis=0)

In [62]:
precision_recall_balance = 0.5
np.percentile(
            avg_similarities, 
            (1 - precision_recall_balance) * 100)

np.float32(0.08978307)

In [67]:
np.percentile(avg_similarities, 100)

np.float32(0.5473508)

In [68]:
np.max(avg_similarities)

np.float32(0.5473508)

In [69]:
max_similarities = np.max(distances, axis=0)


In [72]:
distances

array([[ 0.57724845,  0.56747884,  0.5658605 , ..., -0.14182474,
        -0.14314456, -0.178755  ],
       [ 0.5174532 ,  0.49693608,  0.49479768, ..., -0.1275644 ,
        -0.1365313 , -0.15114349]], dtype=float32)